# Stage 1: Pagination Criteria Notebook

This notebook generates pagination criteria for each table using three methods: naive (full-table fetch), LLM-based (OpenRouter), and statistical cardinality estimation. Results are saved as JSON files in a timestamped folder under `processing/1_pagination/`.

In [23]:
import os
import glob
import json
import pandas as pd
from datetime import datetime

## Locate Latest Data Folder

In [24]:
data_root = 'processing/0_data/'
folders = [f for f in os.listdir(data_root) if os.path.isdir(os.path.join(data_root, f))]
latest_folder = sorted(folders)[-1] if folders else None
data_path = os.path.join(data_root, latest_folder) if latest_folder else None
assert data_path and os.path.exists(data_path), 'No data folder found.'

## Load Table JSONs

In [25]:
json_files = glob.glob(os.path.join(data_path, '*.json'))
tables = []
for jf in json_files:
    with open(jf, 'r') as f:
        obj = json.load(f)
    # Find corresponding CSV
    csv_path = jf.replace('.json', '.csv')
    if not os.path.exists(csv_path):
        print(f'Missing CSV for {jf}')
        continue
    tables.append({'path': jf, 'meta': obj['meta'], 'csv': csv_path})

## Define Pagination Criteria Methods

In [26]:
def naive_criteria(table):
    return [{
        'criteria': 'none',
        'expected_page_size': len(table),
        'estimated_pages': 1,
        'expected_page_variance': 0,
        'pages': ['all']
    }]

def statistical_cardinality_criteria(df):
    # Placeholder: select columns with lowest variance
    variances = df.var(numeric_only=True)
    if not variances.empty:
        col = variances.idxmin()
        return [{
            'criteria': col,
            'expected_page_size': int(df.shape[0] / 5),
            'estimated_pages': 5,
            'expected_page_variance': int(variances.min()),
            'pages': []  # Could be filled with unique values or ranges
        }]

In [27]:
# --- Config ---
RUN_CRITERIA = 'all'  # one of: 'naive', 'statistical', 'llm', 'all'
RUN_MODELS = [
    'google/gemini-2.5-flash-lite',
    'deepseek/deepseek-chat-v3.1',
    'openai/gpt-4o-mini',
 ]  # used only when RUN_CRITERIA in {'llm','all'}

# Restrict which tables to run (None = all). Values are table 'name' from meta.
TABLE_WHITELIST = None  # e.g. ['south_african_class_15f_4_8_2']
TABLE_BLACKLIST = []
TABLE_LIMIT = None  # e.g. 5

# Heuristic targets
PAGE_SIZE_TARGET = 50
MIN_PAGES = 3
MAX_PAGES = 10
MIN_ROWS_THRESHOLD = 100  # only consider tables with >100 rows

# Re-run behavior
SKIP_IF_EXISTS = False  # skip writing if output exists for given criteria/model/table

# OpenRouter config (read API key from env var OPENROUTER_API_KEY)
OPENROUTER_BASE_URL = 'https://openrouter.ai/api/v1'
# OPENROUTER_API_KEY_ENV = 'OPENROUTER_API_KEY'
OPENROUTER_ORIGIN = 'openrouter'  # label only

import os, json, time, re, math, random
from datetime import datetime
import pandas as pd
from pathlib import Path

RUN_TS = datetime.now().strftime('%Y%m%d_%H%M%S')
print('Run timestamp:', RUN_TS)

Run timestamp: 20250903_223321


In [28]:
api_key = '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'

In [29]:
# --- LLM prompt for pagination criteria ---
def build_llm_prompt(meta: dict, csv_path: str) -> str:
    df = pd.read_csv(csv_path)
    cols = list(df.columns)
    # Use 20 random rows instead of head
    if df.shape[0] > 20:
        sample_df = df.sample(n=20, random_state=42)
    else:
        sample_df = df
    head = sample_df.to_string(index=False)
    query = meta.get('query_without_cutoff') or meta.get('table_title') or meta.get('name')
    tmpl = '''
### **Task Explanation**

You are a data scientist specializing in tabular data retrieval and result organization.
Your task is to recommend efficient **pagination strategies** for large tables.

Pagination should:

* Allow balanced, predictable page sizes.
* Be based on available columns whenever possible.
* Include a **complete list of page keys to request**.
* **If the full list of pages cannot be determined, omit the criteria entirely.**
* Keep in mind that the table head is very partial, and the full table may have many more rows and a wider range of values.
* Aim for an **average** page size of around %d rows** and no more than 5 pages. Ranges are acceptable e.g "1960-1965".

---

### **Output Format**

Return up to **3 recommended pagination criteria**, ordered from best to worst.
Output must be **raw JSON** in the following format:

```json
[
{
    "criteria": "comma-separated column names used for pagination",
    "expected_page_size": "approximate number of rows per page and variance, integer ONLY",
    "estimated_pages": "approximate number of total pages, integer ONLY",
    "expected_page_variance": "approximate variance in page size, integer ONLY",
    "pages": ["full list of page keys/ranges to request downstream"]
}
]
```

---

### **Example Input**

```
Query: Retrieve all Nobel Prize winners in Physics  
Columns: year, laureate_name, country  
Table head:  
year | laureate_name   | country  
1901 | Wilhelm Röntgen | Germany  
1902 | Hendrik Lorentz | Netherlands  
1902 | Pieter Zeeman   | Netherlands  
```

---

### **Example Output**

```json
[
{
    "criteria": "year",
    "expected_page_size": "50",
    "expected_page_variance": "10",
    "estimated_pages": 3,
    "pages": ["1901-1920", "1921-1940", "1941-1960"]
},
{
    "criteria": "first_letter_of_laureate_name",
    "expected_page_size": "30",
    "expected_page_variance": "15",
    "estimated_pages": 9,
    "pages": ["A-C", "D-F", "G-I", "J-L", "M-O", "P-R", "S-U", "V-X", "Y-Z"]
}
]
```

---

### **Actual Input**
'''
    query = f'''
### Input
Query: {query}
Columns: {', '.join(cols)}
Table head:
{head}
'''
    return tmpl + query

def strip_code_fences(txt):
    return txt.replace('```json', '').replace('```', '').strip()

def parse_llm_recommendations(txt: str):
    if not txt:
        return []
    s = strip_code_fences(txt)
    try:
        data = json.loads(s)
        if isinstance(data, dict):
            data = [data]
        # sanitize numeric fields if strings
        out = []
        for r in data:
            if not isinstance(r, dict):
                continue
            rec = {
                'criteria': r.get('criteria', ''),
                'expected_page_size': int(str(r.get('expected_page_size', 0)).strip().split()[0] or 0),
                'expected_page_variance': int(str(r.get('expected_page_variance', 0)).strip().split()[0] or 0),
                'estimated_pages': int(str(r.get('estimated_pages', 0)).strip().split()[0] or 0),
                'pages': r.get('pages', []) or []
            }
            out.append(rec)
        return out
    except Exception as e:
        print(f'Failed to parse LLM JSON with {txt}:', e)
        return []

In [30]:
# --- Naive recommendation ---
def naive_recommendation(meta: dict, csv_path: str):
    try:
        df = pd.read_csv(csv_path)
        n = df.shape[0]
        return {
            'criteria': '',
            'estimated_pages': 1,
            'pages': ['ALL'],
            'expected_page_size': n,
            'expected_page_variance': 0
        }
    except Exception as e:
        print('Naive recommendation error:', e)
        return {}

In [31]:
# --- LLM querying logic ---
import requests

def query_llm_for_criteria(meta: dict, df: pd.DataFrame, model: str):
    # api_key = os.environ.get(OPENROUTER_API_KEY_ENV, '')
    if not api_key:
        print('No OpenRouter API key found in environment.')
        return None, None, None
    prompt = build_llm_prompt(meta, df)
    headers = {
        'Authorization': f'Bearer {api_key}',
        'Content-Type': 'application/json'
    }
    data = {
        'model': model,
        'messages': [
            {'role': 'user', 'content': prompt}
        ],
        'max_tokens': 1024,
        'temperature': 0
    }
    try:
        response = requests.post(f'{OPENROUTER_BASE_URL}/chat/completions', headers=headers, json=data)
        response.raise_for_status()
        content = response.json()['choices'][0]['message']['content']
        return prompt, content, parse_llm_recommendations(content)
    except Exception as e:
        print(f'LLM query failed for table {meta.get("name", "?")}, model {model}, query: {prompt}', e)
        return None, None, None

In [32]:
# --- Statistical cardinality recommendation ---
def get_llm_page_keys(meta, col, df, model='openai/gpt-4o-mini'):
    head = df.head(20).to_string(index=False)
    query = meta.get('query_without_cutoff') or meta.get('table_title') or meta.get('name')
    prompt = f'''
Given the following table and the column(s) [{col}],
enumerate all possible values (or value ranges) for these columns that would be useful for pagination.
Return a JSON list of values or ranges, suitable for requesting pages downstream.
If the column is numeric, suggest reasonable ranges or bins.
If categorical, list all unique values. If the list is too long, summarize or group as needed.

Table head:
{head}

Column(s): {col}
Query: {query}
Return raw JSON only: ["value1", "value2", ...]
'''
    import requests, os
    headers = {'Authorization': f'Bearer {api_key}', 'Content-Type': 'application/json'}
    data = {
        'model': model,
        'messages': [{'role': 'user', 'content': prompt}],
        'max_tokens': 512,
        'temperature': 0
    }
    try:
        response = requests.post('https://openrouter.ai/api/v1/chat/completions', headers=headers, json=data)
        response.raise_for_status()
        content = response.json()['choices'][0]['message']['content']
        # Remove code fences if present
        content = content.replace('```json', '').replace('```', '').strip()
        page_keys = json.loads(content)
        if isinstance(page_keys, dict):
            page_keys = list(page_keys.values())
        return page_keys
    except Exception as e:
        print(f'LLM page key query failed for column {col}:', e)
        return []

def statistical_cardinality_recommendation(meta: dict, csv_path: str, sample_size: int = 100):
    try:
        df = pd.read_csv(csv_path)
        n = df.shape[0]
        if n == 0:
            return {}
        sample_n = min(sample_size, n)
        df_sample = df.sample(n=sample_n, random_state=42) if n > sample_n else df
        variances = df_sample.var(numeric_only=True)
        if not variances.empty:
            col = variances.idxmin()
            # Query LLM for page keys if none found
            page_keys = get_llm_page_keys(meta, col, df)
            return {
                'criteria': col,
                'estimated_pages': min(max(int(n / PAGE_SIZE_TARGET), MIN_PAGES), MAX_PAGES),
                'pages': page_keys,
                'expected_page_size': int(n / max(1, min(max(int(n / PAGE_SIZE_TARGET), MIN_PAGES), MAX_PAGES))),
                'expected_page_variance': int(variances.min())
            }
    except Exception as e:
        print('Statistical cardinality error:', e)
    return {}

In [33]:
output_root = 'processing/1_pagination/'
timestamp = RUN_TS
output_folder = os.path.join(output_root, timestamp)
os.makedirs(output_folder, exist_ok=True)

table_iter = tables[:TABLE_LIMIT] if TABLE_LIMIT else tables
for table in table_iter:
    meta = table['meta']
    csv_path = table['csv']
    print(f'Processing table: {meta.get("name", "?")}, CSV: {csv_path}')
    if TABLE_WHITELIST and meta.get('name') not in TABLE_WHITELIST:
        continue
    if TABLE_BLACKLIST and meta.get('name') in TABLE_BLACKLIST:
        continue
    try:
        df = pd.read_csv(csv_path)
    except Exception as e:
        print(f'Error loading CSV {csv_path}:', e)
        continue
    if df.shape[0] < MIN_ROWS_THRESHOLD:
        continue
    result = {}
    print(f'Processing table: {meta.get("name", "?")}, CSV: {csv_path}')
    if RUN_CRITERIA in {'naive', 'all'}:
        result['naive'] = naive_recommendation(meta, csv_path)
    if RUN_CRITERIA in {'statistical', 'all'}:
        result['statistical'] = statistical_cardinality_recommendation(meta, csv_path)
    if RUN_CRITERIA in {'llm', 'all'}:
        result['llm'] = {}
        for model in RUN_MODELS:
            llm_result = None
            for attempt in range(3):
                query, response, llm_result = query_llm_for_criteria(meta, csv_path, model)
                # Accept result if not None and not empty list
                if llm_result is not None and (not isinstance(llm_result, list) or len(llm_result) > 0):
                    break
                print(f'LLM retry {attempt+1} for model {model} on table {meta.get("name", "?")} with query {query} and result {llm_result}')
            if llm_result is not None and (not isinstance(llm_result, list) or len(llm_result) > 0):
                result['llm'][model] = llm_result
            else:
                print(f'Skipping table {meta.get("name", "?")} for model {model} due to repeated LLM failure.')
    out_payload = {'meta': meta, 'pagination_criteria': result}
    out_path = os.path.join(output_folder, os.path.basename(table['path']))
    with open(out_path, 'w') as f:
        json.dump(out_payload, f, indent=2)
    # Copy CSV file to output folder
    out_csv_path = os.path.join(output_folder, os.path.basename(csv_path))
    df.to_csv(out_csv_path, index=False)
print('Done. Results saved to', output_folder)

Processing table: english_latin_rivalry_1887_2012, CSV: processing/0_data/20250903_223317/25_english_latin_rivalry_1887_2012.csv
Processing table: english_latin_rivalry_1887_2012, CSV: processing/0_data/20250903_223317/25_english_latin_rivalry_1887_2012.csv
Processing table: australia_demographics_1900_2010, CSV: processing/0_data/20250903_223317/3_australia_demographics_1900_2010.csv
Processing table: australia_demographics_1900_2010, CSV: processing/0_data/20250903_223317/3_australia_demographics_1900_2010.csv
Processing table: elements, CSV: processing/0_data/20250903_223317/15_elements.csv
Processing table: elements, CSV: processing/0_data/20250903_223317/15_elements.csv
Processing table: rock_band_downloadable_2011, CSV: processing/0_data/20250903_223317/12_rock_band_downloadable_2011.csv
Processing table: rock_band_downloadable_2011, CSV: processing/0_data/20250903_223317/12_rock_band_downloadable_2011.csv
Processing table: living_proof_the_farewell_tour, CSV: processing/0_data/2